# 🏋️‍♂️ Fitness Tracker Project using Python + SQL

This notebook helps track workouts and meals using SQLite for storage and pandas/matplotlib for analysis.

In [ ]:
# 📦 Setup
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

# Connect to SQLite
conn = sqlite3.connect("fitness_tracker.db")
cursor = conn.cursor()

In [ ]:
# 🧱 Create Tables
cursor.execute('''
CREATE TABLE IF NOT EXISTS workouts (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    date TEXT,
    workout_type TEXT,
    duration_mins INTEGER,
    calories_burned INTEGER
)''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS meals (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    date TEXT,
    meal TEXT,
    calories INTEGER,
    protein INTEGER,
    carbs INTEGER,
    fat INTEGER
)''')
conn.commit()

In [ ]:
# 📊 Sample Data (You can replace this with CSV import if needed)
workouts_data = [
    ("2025-05-01", "Cardio", 45, 350),
    ("2025-05-02", "Strength", 60, 400),
    ("2025-05-03", "Yoga", 30, 150)
]

meals_data = [
    ("2025-05-01", "Oatmeal & Eggs", 450, 25, 50, 10),
    ("2025-05-01", "Salad & Chicken", 600, 40, 30, 20),
    ("2025-05-02", "Protein Shake", 300, 30, 10, 5)
]

In [ ]:
# 💾 Insert Data
cursor.executemany("INSERT INTO workouts (date, workout_type, duration_mins, calories_burned) VALUES (?, ?, ?, ?)", workouts_data)
cursor.executemany("INSERT INTO meals (date, meal, calories, protein, carbs, fat) VALUES (?, ?, ?, ?, ?, ?)", meals_data)
conn.commit()

In [ ]:
# 📈 Query and Analyze
workouts_df = pd.read_sql_query("SELECT * FROM workouts", conn)
meals_df = pd.read_sql_query("SELECT * FROM meals", conn)

display(workouts_df)
display(meals_df)

daily_intake = meals_df.groupby('date')['calories'].sum()
daily_burned = workouts_df.groupby('date')['calories_burned'].sum()

summary_df = pd.DataFrame({
    "Calories Consumed": daily_intake,
    "Calories Burned": daily_burned
}).fillna(0)
summary_df["Net Calories"] = summary_df["Calories Consumed"] - summary_df["Calories Burned"]
summary_df

In [ ]:
# 📉 Visualize
summary_df.plot(kind='bar', figsize=(10, 5), title="Calories Consumed vs Burned")
plt.ylabel("Calories")
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# ➕ Optional Input Function (Run only if needed)
def add_meal():
    date = input("Enter date (YYYY-MM-DD): ")
    meal = input("Meal description: ")
    cal = int(input("Calories: "))
    prot = int(input("Protein (g): "))
    carb = int(input("Carbs (g): "))
    fat = int(input("Fat (g): "))
    cursor.execute("INSERT INTO meals (date, meal, calories, protein, carbs, fat) VALUES (?, ?, ?, ?, ?, ?)",
                   (date, meal, cal, prot, carb, fat))
    conn.commit()
    print("Meal added successfully!")

# add_meal()